# Preprocessing Pipelines 

# Librerias

In [114]:
# general
import numpy as np
import pandas as pd
import re
import os
import timeit
import joblib


# visualización
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import warn
plt.style.use('seaborn-colorblind')
#from data_exploration import explore


# preprocesamiento
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA

# ML
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error
import pylab
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

# ML pipelines
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics.pairwise import rbf_kernel


from sklearn import set_config

C:\Users\Diego\AppData\Local\Temp\ipykernel_16936\3087161504.py:14: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')


# Cargar Bases de datos (Tablas)

In [4]:
# cargar base de datos
data = pd.read_csv('./data/titanic.csv')
data.head(3)
print(data.shape)
data

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Preparar base de datos para Pipelines

## Separar y dependiente, x variables

In [5]:
x = data.drop("Survived", axis=1)
y = data["Survived"]
x,y

(     PassengerId  Pclass                                               Name  \
 0              1       3                            Braund, Mr. Owen Harris   
 1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
 2              3       3                             Heikkinen, Miss. Laina   
 3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
 4              5       3                           Allen, Mr. William Henry   
 ..           ...     ...                                                ...   
 886          887       2                              Montvila, Rev. Juozas   
 887          888       1                       Graham, Miss. Margaret Edith   
 888          889       3           Johnston, Miss. Catherine Helen "Carrie"   
 889          890       1                              Behr, Mr. Karl Howell   
 890          891       3                                Dooley, Mr. Patrick   
 
         Sex   Age  SibSp  Parch      

## Separar bases de datos (tablas) en datos de entrenamiento y datos de prueba

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


## Extraer tipos de variables

In [7]:
# extraer nombres de columnas por tipo 
# retorna tupla de listas con nombres de columnas por tipo

def columns_types(data):
    
    # definir estrucutra de datos por tipo de variable
    numeric_types=['int', 'int32', 'int64', 'float', 'float32', 'float64']
    category_types = ['object','category']
    str_types = ['string']
    bool_types = ['bool']
    date_types = ['datetime64', 'datetime', 'datetime64[ns]']
    
    # listas de nombres de variables
    numeric_columns = []
    str_columns = []
    bool_columns = []
    date_columns = []
    category_columns = []

    for column in data.columns:
        
        col_type = str(data[column].dtype)
        
        if any(col_type.startswith(t) for t in numeric_types) or any(col_type.startswith(t) for t in numeric_types):
            numeric_columns.append(column)
        elif any(col_type.startswith(t) for t in str_types):
            str_columns.append(column)
        elif any(col_type.startswith(t) for t in bool_types):
            bool_columns.append(column)
        elif any(col_type.startswith(t) for t in date_types):
            date_columns.append(column)
        elif any(col_type.startswith(t) for t in category_types):
            category_columns.append(column)
            

    return numeric_columns, category_columns, str_columns,bool_columns, date_columns



In [8]:
# extraer nombres de columnas por tipo 
numeric_columns, category_columns, str_columns,bool_columns, date_columns = columns_types(data)
numeric_columns, category_columns, str_columns,bool_columns, date_columns



(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
 ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'],
 [],
 [],
 [])

# Pipelines de preprocesamiento

## Pipeline de Tratamiento de NA (Missing Values)

In [9]:
# crear pipeline de tratamiento Missing Values
# crear SimpleImputer
# imputacion simple, estrategia: mediana
imputer = SimpleImputer(strategy="median")
print('Tipo de Imputación de Missing Value NA:  ',imputer.strategy)
imputer

Tipo de Imputación de Missing Value NA:   median


SimpleImputer(strategy='median')

In [10]:
# extraer columnas numericas de base de datos
columnas_numericas1 = data.select_dtypes(include=[np.number])
print(type(columnas_numericas1))
columnas_numericas1

<class 'pandas.core.frame.DataFrame'>


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000
887,888,1,1,19.0,0,0,30.0000
888,889,0,3,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


In [11]:
# transformar DataFrame de variables numéricas con pipeline de Missing Values
var_num_preprocess1 = imputer.fit_transform(columnas_numericas1)
var_num_preprocess1

array([[  1.    ,   0.    ,   3.    , ...,   1.    ,   0.    ,   7.25  ],
       [  2.    ,   1.    ,   1.    , ...,   1.    ,   0.    ,  71.2833],
       [  3.    ,   1.    ,   3.    , ...,   0.    ,   0.    ,   7.925 ],
       ...,
       [889.    ,   0.    ,   3.    , ...,   1.    ,   2.    ,  23.45  ],
       [890.    ,   1.    ,   1.    , ...,   0.    ,   0.    ,  30.    ],
       [891.    ,   0.    ,   3.    , ...,   0.    ,   0.    ,   7.75  ]])

In [12]:
# ver columnas transformadas por pipeline(imputer)
imputer.feature_names_in_

# crear DataFrame con variables numericas preprocesadas por pipeline(imputer)
data_preprocess1 = pd.DataFrame(var_num_preprocess1, columns=columnas_numericas1.columns,
                          index=columnas_numericas1.index)
data_preprocess1

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500
...,...,...,...,...,...,...,...
886,887.0,0.0,2.0,27.0,0.0,0.0,13.0000
887,888.0,1.0,1.0,19.0,0.0,0.0,30.0000
888,889.0,0.0,3.0,28.0,1.0,2.0,23.4500
889,890.0,1.0,1.0,26.0,0.0,0.0,30.0000


In [13]:
numeric_columns, category_columns, str_columns,bool_columns, date_columns

(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'],
 ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'],
 [],
 [],
 [])

In [14]:
# quitar columna 'Sex'
#str_columns.remove('Sex')
#str_columns

## Variables numéricas 

In [15]:
# crear pipeline de variables numericas
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="mean")),
    ("standardize", StandardScaler()),
])

# crear pipeline de preprocesamiento de variables numericas
# pasos: imputacion simple por mediana, escalamiento estandar
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
num_pipeline

# ver pipeline
set_config(display='diagram')

# aplicar pipeline de preprocesamiento a variables numericas
numeric_preprocess1 = num_pipeline.fit_transform(data[numeric_columns])

# convertir columnas numericas preprocesadas a DataFrame (tabla)
df_numeric_preprocess = pd.DataFrame(numeric_preprocess1, columns=num_pipeline.get_feature_names_out())
df_numeric_preprocess

# ver pasos de preprocesamiento de pipeline de variables numericas
num_pipeline.steps
num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [16]:
# ver paso de pipeline
num_pipeline.named_steps["simpleimputer"]

# cambiar parametros de paso de pipeline
num_pipeline.set_params(simpleimputer__strategy="median")



Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

## Variables categóricas

In [17]:
category_columns

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [18]:
data[category_columns]


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S
...,...,...,...,...,...
886,"Montvila, Rev. Juozas",male,211536,NaN,S
887,"Graham, Miss. Margaret Edith",female,112053,B42,S
888,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,NaN,S
889,"Behr, Mr. Karl Howell",male,111369,C148,C


In [19]:
data_copy = data.copy(deep=True)
data_copy = data.dropna()
data_copy


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [20]:
category_columns.remove('Name')
category_columns.remove('Ticket')

In [21]:
# crear codificador ordinal (categorias por orden)
ordinal_encoder = OrdinalEncoder()

# transformar variables categoricas 
columnas_cate_codificada1 = ordinal_encoder.fit_transform(data[category_columns])

# ver categorias de codificador
ordinal_encoder.categories_

[array(['female', 'male'], dtype=object),
 array(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31',
        'A32', 'A34', 'A36', 'A5', 'A6', 'A7', 'B101', 'B102', 'B18',
        'B19', 'B20', 'B22', 'B28', 'B3', 'B30', 'B35', 'B37', 'B38',
        'B39', 'B4', 'B41', 'B42', 'B49', 'B5', 'B50', 'B51 B53 B55',
        'B57 B59 B63 B66', 'B58 B60', 'B69', 'B71', 'B73', 'B77', 'B78',
        'B79', 'B80', 'B82 B84', 'B86', 'B94', 'B96 B98', 'C101', 'C103',
        'C104', 'C106', 'C110', 'C111', 'C118', 'C123', 'C124', 'C125',
        'C126', 'C128', 'C148', 'C2', 'C22 C26', 'C23 C25 C27', 'C30',
        'C32', 'C45', 'C46', 'C47', 'C49', 'C50', 'C52', 'C54', 'C62 C64',
        'C65', 'C68', 'C7', 'C70', 'C78', 'C82', 'C83', 'C85', 'C86',
        'C87', 'C90', 'C91', 'C92', 'C93', 'C95', 'C99', 'D', 'D10 D12',
        'D11', 'D15', 'D17', 'D19', 'D20', 'D21', 'D26', 'D28', 'D30',
        'D33', 'D35', 'D36', 'D37', 'D45', 'D46', 'D47', 'D48', 'D49',
        'D50', 'D56', 'D6',

In [22]:
# crear codificador One-Hot (0-1)
onehot_encoder = OneHotEncoder()

# transformar variables categoricas 
columnas_cate_codificada2 = onehot_encoder.fit_transform(data[category_columns])
# ver categorias de codificador
onehot_encoder.categories_

[array(['female', 'male'], dtype=object),
 array(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31',
        'A32', 'A34', 'A36', 'A5', 'A6', 'A7', 'B101', 'B102', 'B18',
        'B19', 'B20', 'B22', 'B28', 'B3', 'B30', 'B35', 'B37', 'B38',
        'B39', 'B4', 'B41', 'B42', 'B49', 'B5', 'B50', 'B51 B53 B55',
        'B57 B59 B63 B66', 'B58 B60', 'B69', 'B71', 'B73', 'B77', 'B78',
        'B79', 'B80', 'B82 B84', 'B86', 'B94', 'B96 B98', 'C101', 'C103',
        'C104', 'C106', 'C110', 'C111', 'C118', 'C123', 'C124', 'C125',
        'C126', 'C128', 'C148', 'C2', 'C22 C26', 'C23 C25 C27', 'C30',
        'C32', 'C45', 'C46', 'C47', 'C49', 'C50', 'C52', 'C54', 'C62 C64',
        'C65', 'C68', 'C7', 'C70', 'C78', 'C82', 'C83', 'C85', 'C86',
        'C87', 'C90', 'C91', 'C92', 'C93', 'C95', 'C99', 'D', 'D10 D12',
        'D11', 'D15', 'D17', 'D19', 'D20', 'D21', 'D26', 'D28', 'D30',
        'D33', 'D35', 'D36', 'D37', 'D45', 'D46', 'D47', 'D48', 'D49',
        'D50', 'D56', 'D6',

# Transformador Personalizado


In [39]:
# crear transformador logaritmico
# aplicar función logaritmica
# transformar variable numerica 'Fare'
log_transformer = FunctionTransformer(np.log, inverse_func=np.exp)
log_pop = log_transformer.transform(data[["Fare"]])

# crear transformador RBF
# función RBF
rbf_transformer = FunctionTransformer(rbf_kernel,
                                      kw_args=dict(Y=[[35.]], gamma=0.1))
fare_simil_35 = rbf_transformer.transform(data[["Fare"]])
fare_simil_35



c:\Users\Diego\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


array([[3.60255444e-034],
       [6.70028126e-058],
       [1.45813829e-032],
       [5.91668541e-015],
       [2.86472677e-032],
       [2.54455045e-031],
       [4.47817494e-013],
       [3.79127865e-009],
       [1.82702769e-025],
       [8.80631363e-002],
       [2.85701220e-015],
       [7.92553933e-004],
       [2.86472677e-032],
       [2.49683166e-001],
       [9.93299759e-033],
       [2.09879105e-016],
       [3.16960726e-002],
       [9.55316054e-022],
       [2.81118530e-013],
       [3.13563173e-034],
       [3.03539138e-004],
       [9.55316054e-022],
       [2.56079488e-032],
       [9.75309912e-001],
       [3.79127865e-009],
       [2.71168298e-001],
       [3.13563173e-034],
       [0.00000000e+000],
       [1.13762609e-032],
       [1.24477801e-032],
       [4.99821083e-003],
       [0.00000000e+000],
       [5.63534330e-033],
       [8.54031174e-027],
       [2.32173966e-097],
       [2.81118530e-013],
       [3.20964358e-034],
       [2.86472677e-032],
       [2.81

# Transformador de Columnas

In [46]:
data_copy = data.copy(deep=True)
data_copy

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [95]:
# crear pipeline de preprocesamiento
# imputador simple por mediana
# estandarización estandar
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
    ])

# hacer pipeline
# imputación simple por mediana
# estandarización estandar
num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler())

# mostrar pipeline
set_config(display='diagram')

# transformar variables numericas (columnas) en DataFrame
data_copy[numeric_columns] = num_pipeline.fit_transform(data[numeric_columns])
data_copy
  

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,-1.730108,-0.789272,0.827377,"Braund, Mr. Owen Harris",male,-0.565736,0.432793,-0.473674,A/5 21171,-0.502445,NaN,S
1,-1.726220,1.266990,-1.566107,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.663861,0.432793,-0.473674,PC 17599,0.786845,C85,C
2,-1.722332,1.266990,0.827377,"Heikkinen, Miss. Laina",female,-0.258337,-0.474545,-0.473674,STON/O2. 3101282,-0.488854,NaN,S
3,-1.718444,1.266990,-1.566107,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.433312,0.432793,-0.473674,113803,0.420730,C123,S
4,-1.714556,-0.789272,0.827377,"Allen, Mr. William Henry",male,0.433312,-0.474545,-0.473674,373450,-0.486337,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,1.714556,-0.789272,-0.369365,"Montvila, Rev. Juozas",male,-0.181487,-0.474545,-0.473674,211536,-0.386671,NaN,S
887,1.718444,1.266990,-1.566107,"Graham, Miss. Margaret Edith",female,-0.796286,-0.474545,-0.473674,112053,-0.044381,B42,S
888,1.722332,-0.789272,0.827377,"Johnston, Miss. Catherine Helen ""Carrie""",female,-0.104637,0.432793,2.008933,W./C. 6607,-0.176263,NaN,S
889,1.726220,1.266990,-1.566107,"Behr, Mr. Karl Howell",male,-0.258337,-0.474545,-0.473674,111369,-0.044381,C148,C


In [96]:
# ver base de datos(tabla) en columnas transformadas
data_copy[numeric_columns]

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,-1.730108,-0.789272,0.827377,-0.565736,0.432793,-0.473674,-0.502445
1,-1.726220,1.266990,-1.566107,0.663861,0.432793,-0.473674,0.786845
2,-1.722332,1.266990,0.827377,-0.258337,-0.474545,-0.473674,-0.488854
3,-1.718444,1.266990,-1.566107,0.433312,0.432793,-0.473674,0.420730
4,-1.714556,-0.789272,0.827377,0.433312,-0.474545,-0.473674,-0.486337
...,...,...,...,...,...,...,...
886,1.714556,-0.789272,-0.369365,-0.181487,-0.474545,-0.473674,-0.386671
887,1.718444,1.266990,-1.566107,-0.796286,-0.474545,-0.473674,-0.044381
888,1.722332,-0.789272,0.827377,-0.104637,0.432793,2.008933,-0.176263
889,1.726220,1.266990,-1.566107,-0.258337,-0.474545,-0.473674,-0.044381


In [57]:
# ver paso de Simpleimputer
num_pipeline.named_steps["simpleimputer"] 


SimpleImputer(strategy='median')

In [67]:
# cambiar estrategia en paso
num_pipeline.set_params(simpleimputer__strategy="median")


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [97]:
# hacer pipeline para variables categoricas
# imputación simple por moda
# codificación one-hot [0-1] 

cate_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

# transformar variables numericas 
data_copy[category_columns] = cate_pipeline.fit_transform(data_copy[category_columns])
data_copy[category_columns]


ValueError: Columns must be same length as key

In [74]:
numeric_columns

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [73]:
category_columns

['Sex', 'Cabin', 'Embarked']

In [90]:

# crear copia de base de datos (DataFrame)
data_copy2 = data.copy(deep=True)

# crear Pipeline de preprocesamiento
# num_pipeline: variables numericas
# cat_pipeline: variables categoricas
preprocessing1 = ColumnTransformer([
    ("numericas", num_pipeline, numeric_columns),
    ("categoricas", cate_pipeline, category_columns),
])

# hacer pipeline de preprocesamiento
# make_column_selector(dtype_include=tipo) seleccionar columnas por tipo
# num_pipeline, cat_pipeline
preprocessing1 = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cate_pipeline, make_column_selector(dtype_include=object)),
)

# transformar base de datos con pipeline de preprocesamiento
data_copy2_preprocess = preprocessing1.fit_transform(data_copy2)


# crear DataFrame con base de datos preprocesada
"""
data_copy2_preprocess = pd.DataFrame(
    data_copy2_preprocess,
    columns=preprocessing1.get_feature_names_out(),
    index=data_copy2.index)
data_copy2_preprocess 

"""


data_copy2_preprocess

# transformar base de datos con Pipeline de preprocesamiento
#data_preprocess1 = preprocessing.fit_transform(data)
# crear DataFrame
#data_preprocess1 = pd.DataFrame(
#    data_preprocess1,
#    columns=preprocessing.get_feature_names_out()
#    )
#preprocessing.get_feature_names_out()


<891x1731 sparse matrix of type '<class 'numpy.float64'>'
	with 10691 stored elements in Compressed Sparse Row format>

In [29]:
# crear ColumnTransformer
# pipeline1:   num_pipeline - imputacion simple por mediana, escalamiento estandar
# pipeline2: cate_pipeline - imputacion simple por moda(clase más frecuente), codificación one-hot 
preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cate_pipeline, make_column_selector(dtype_include=object)),
)

# transformar base de datos con ColumnTransformer
data_preprocess1 = preprocessing.fit_transform(data)


# extra code – shows that we can get a DataFrame out if we want
df_preprocess1 = pd.DataFrame(
    data_preprocess1,
    columns=preprocessing.get_feature_names_out(),
    index=data.index)
df_preprocess1.head(2)


ValueError: Shape of passed values is (891, 1), indices imply (891, 1731)

In [103]:
# crear lista de pasos en pipeline de preprocesamiento
steps1 = [
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2))
]

pipeline1 = Pipeline(steps1)
pipeline1

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2))])

# Guardar Bases de datos Preprocesadas

In [110]:
data_copy.to_csv('data/base1.csv')


# Guardar Pipelines de preprocesamiento

In [109]:
joblib.dump(pipeline1, 'data/pipeline_preprocess1.pkl')
joblib.dump(preprocessing1, 'data/pipeline_preprocess2.pkl')


['data/pipeline_preprocess2.pkl']

In [5]:
numeric_features = ["Age", "Fare"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

## Variables categóricas

In [6]:
categorical_features = ["Pclass", "Sex", "Embarked"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

## Variables de texto

In [7]:
text_features = ["Name"]
text_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='')),
    ('tfidf', TfidfVectorizer())
])

# Pipeline de Preprocesamiento

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('text', text_transformer, text_features)
    ])

# Entrenar Pipeline de Preprocesamiento

In [12]:
# Apply preprocessing to the training data
x_train_preprocessed = preprocessor.fit_transform(x_train)


AttributeError: 'numpy.ndarray' object has no attribute 'lower'